In [15]:
!git clone https://github.com/cse151a-DrugReviewAnalysis/DrugReviewAnalysis.git

fatal: destination path 'DrugReviewAnalysis' already exists and is not an empty directory.


In [16]:
!python3 -m pip install nltk

In [17]:
import nltk

nltk.download(["vader_lexicon"])



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [18]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

#sia.polarity_scores("Wow, NLTK is really powerful!")["compound"]

In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [20]:
import pandas as pd

df1 = pd.read_csv('/content/DrugReviewAnalysis/drugsComTrain_raw.tsv', sep="\t")

df_train = pd.read_csv('/content/DrugReviewAnalysis/drugsTrain_processed.csv', sep="\t")

df_test = pd.read_csv('/content/DrugReviewAnalysis/drugsTest_processed.csv', sep="\t")

In [21]:
# WARNING: takes a minute
review_score_column = df_train['processed_review'].apply(sia.polarity_scores)
df_train['compound_score'] = review_score_column

review_score_column = df_test['processed_review'].apply(sia.polarity_scores)
df_test['compound_score'] = review_score_column



In [22]:
# save positive and negative
df_train['positive_score'] = df_train['compound_score'].apply(lambda x: x['pos'])
df_test['positive_score'] = df_test['compound_score'].apply(lambda x: x['pos'])
df_train['negative_score'] = df_train['compound_score'].apply(lambda x: x['neg'])
df_test['negative_score'] = df_test['compound_score'].apply(lambda x: x['neg'])

df_train['compound_score'] = df_train['compound_score'].apply(lambda x: x['compound'])
df_test['compound_score'] = df_test['compound_score'].apply(lambda x: x['compound'])

In [23]:
X_train = df_train.drop(columns=["rating"])
y_train = df_train["rating"]

X_test = df_test.drop(columns=["rating"])
y_test = df_test["rating"]

In [24]:
# makes data smaller
X_train = X_train[0:100000]
y_train = y_train[0:100000]
X_test = X_test[0:100000]
y_test = y_test[0:100000]

In [25]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse = False)

encoded_data1 = encoder.fit_transform(X_train[['drugName','condition']])

encoded_columns = encoder.get_feature_names_out(['drugName', 'condition'])

df_encoded_train = pd.DataFrame(encoded_data1, columns=encoded_columns)

encoded_data2 = encoder.fit_transform(X_test[['drugName','condition']])

encoded_columns = encoder.get_feature_names_out(['drugName', 'condition'])

df_encoded_test = pd.DataFrame(encoded_data2, columns=encoded_columns)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [26]:
df_encoded_train

,drugName_a + d cracked skin relief,drugName_a / b otic,drugName_abacavir / dolutegravir / lamivudine,drugName_abacavir / lamivudine / zidovudine,drugName_abatacept,drugName_abilify,drugName_abilify discmelt,drugName_abilify maintena,drugName_abiraterone,drugName_abobotulinumtoxina,...,condition_warts,condition_wegener's granulomatosis,condition_weight loss,condition_wilson's disease,condition_wolff-parkinson-white syndrome,condition_women (minoxidil),condition_women (oxybutynin),condition_wound cleansing,condition_xerostomia,condition_zen shoulde
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
df_encoded_train['positive_score'] = X_train['positive_score']
df_encoded_train['negative_score'] = X_train['negative_score']
df_encoded_train['compound_score'] = X_train['compound_score']
df_encoded_test['positive_score'] = X_train['positive_score']
df_encoded_test['negative_score'] = X_train['negative_score']
df_encoded_test['compound_score'] = X_train['compound_score']

In [28]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, precision_score, recall_score
from keras.utils import to_categorical

In [29]:
smaller_X = df_encoded_train[0:10000]
smaller_y = y_train[0:10000]

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(smaller_X, smaller_y, test_size=0.2, random_state=21)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 3775)
(2000, 3775)
(8000,)
(2000,)


In [31]:
model = Sequential()
model.add(Dense(units = 4000, activation = 'relu', input_dim = X_train.shape[1]))
model.add(Dense(units = 2000, activation = 'relu'))
# model.add(Dense(units = 1000, activation = 'relu'))
# model.add(Dense(units = 500, activation = 'relu'))
# model.add(Dense(units = 250, activation = 'relu'))
model.add(Dense(1))



In [33]:
model.compile(optimizer = "rmsprop", loss = "mse")

model.fit(X_train,y_train,epochs = 6, batch_size= 500, validation_split = 0.2)

yhat = model.predict(X_test)

Epoch 1/6
13/13 [==============================] - 21s 1s/step - loss: 16.3261 - val_loss: 9.9342
Epoch 2/6
13/13 [==============================] - 19s 1s/step - loss: 9.2169 - val_loss: 10.5167
Epoch 3/6
13/13 [==============================] - 24s 2s/step - loss: 8.0187 - val_loss: 10.4406
Epoch 4/6
13/13 [==============================] - 21s 2s/step - loss: 7.7255 - val_loss: 10.4194
Epoch 5/6
13/13 [==============================] - 19s 2s/step - loss: 7.3406 - val_loss: 10.2721
Epoch 6/6
63/63 [==============================] - 5s 71ms/step


In [34]:
y_test_formatted = [[entry] for entry in y_test.to_numpy()]
mse = sum((y_test_formatted - yhat) ** 2 / yhat.size)
print(mse)

[9.6741797]


In [35]:
print(mean_squared_error(y_train, model.predict(X_train)))

250/250 [==============================] - 12s 49ms/step
6.408446640685858
